# ETL Processes

Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()


In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('data/song_data')
song_files

['/home/workspace/data/song_data/A/B/C/TRABCRU128F423F449.json',
 '/home/workspace/data/song_data/A/B/C/TRABCTK128F934B224.json',
 '/home/workspace/data/song_data/A/B/C/TRABCUQ128E0783E2B.json',
 '/home/workspace/data/song_data/A/B/C/TRABCIX128F4265903.json',
 '/home/workspace/data/song_data/A/B/C/TRABCYE128F934CE1D.json',
 '/home/workspace/data/song_data/A/B/C/TRABCEC128F426456E.json',
 '/home/workspace/data/song_data/A/B/C/TRABCXB128F4286BD3.json',
 '/home/workspace/data/song_data/A/B/C/TRABCAJ12903CDFCC2.json',
 '/home/workspace/data/song_data/A/B/C/TRABCPZ128F4275C32.json',
 '/home/workspace/data/song_data/A/B/C/TRABCKL128F423A778.json',
 '/home/workspace/data/song_data/A/B/C/TRABCEI128F424C983.json',
 '/home/workspace/data/song_data/A/B/C/TRABCFL128F149BB0D.json',
 '/home/workspace/data/song_data/A/B/B/TRABBBV128F42967D7.json',
 '/home/workspace/data/song_data/A/B/B/TRABBLU128F93349CF.json',
 '/home/workspace/data/song_data/A/B/B/TRABBXU128F92FEF48.json',
 '/home/workspace/data/so

In [6]:
filepath = 'data/song_data/A/A/A/.json'

In [7]:
df_song = pd.read_json('/home/workspace/data/song_data/A/B/C/TRABCRU128F423F449.json', lines=True)
df_song

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR8IEZO1187B99055E,NaN,,NaN,Marc Shaiman,149.86404,1,SOINLJW12A8C13314C,City Slickers,2008


In [8]:
#df = pd.read_json(song_files, lines=True)
df_song.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR8IEZO1187B99055E,NaN,,NaN,Marc Shaiman,149.86404,1,SOINLJW12A8C13314C,City Slickers,2008


In [9]:
df_song.values

array([['AR8IEZO1187B99055E', nan, '', nan, 'Marc Shaiman', 149.86404, 1,
        'SOINLJW12A8C13314C', 'City Slickers', 2008]], dtype=object)

In [10]:
df1=df_song.loc[:,['song_id', 'title', 'artist_id', 'year', 'duration']]
songdata=df1.values

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [11]:
song_table_insert

'INSERT INTO songs(song_id,title,artist_id,year,duration)values(%s,%s,%s,%s,%s)'

In [12]:
song_data=songdata[0].tolist()
song_data

['SOINLJW12A8C13314C', 'City Slickers', 'AR8IEZO1187B99055E', 2008, 149.86404]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [13]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [14]:
df_song

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR8IEZO1187B99055E,NaN,,NaN,Marc Shaiman,149.86404,1,SOINLJW12A8C13314C,City Slickers,2008


In [15]:
df_artist = df_song.loc[:,['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]
df_artist

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,AR8IEZO1187B99055E,Marc Shaiman,,NaN,NaN


In [16]:
df_artist

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,AR8IEZO1187B99055E,Marc Shaiman,,NaN,NaN


In [17]:
artistdata = df_artist.values


In [18]:
artistdata

array([['AR8IEZO1187B99055E', 'Marc Shaiman', '', nan, nan]], dtype=object)

In [19]:
artist_table_insert

'INSERT INTO artists(artist_id,name,location,latitude,longitude)values(%s,%s,%s,%s,%s);\n'

In [20]:
artist_data=artistdata[0].tolist()

In [21]:
artist_data

['AR8IEZO1187B99055E', 'Marc Shaiman', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [22]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [23]:
log_files = get_files('data/log_data')

In [24]:
filepath = 'data/song_data/2018'

In [25]:
df_log = pd.read_json('/home/workspace/data/log_data/2018/11/2018-11-01-events.json', lines=True)
df_log.head(2)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [26]:
df_log['ts_val']=pd.to_datetime(df_log["ts"])

In [27]:
df_log.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId', 'ts_val'],
      dtype='object')

In [28]:
df_fltr=df_log[df_log.loc[:,'page']=="NextSong"]
df_fltr.loc[2:4,['page','artist']]

,page,artist
2,NextSong,Des'ree
4,NextSong,Mr Oizo


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [29]:
df_fltr["ts"]=pd.to_datetime(df_fltr["ts"])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
#df_fltr["ts"]

In [31]:
df_fltr["hour"]=df_fltr["ts"].dt.hour
#df_fltr["hour"]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
df_fltr["day"]=df_fltr["ts"].dt.day
#df_fltr["day"]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [33]:
df_fltr["weekofyear"]=df_fltr["ts"].dt.week
#df_fltr["weekofyear"]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
df_fltr["month"]=df_fltr["ts"].dt.month
#df_fltr["month"]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
df_fltr["year"]=df_fltr["ts"].dt.year
#df_fltr["year"]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [36]:
#df_fltr["stime"]=df_fltr["ts"].dt.start_time
#df["stime"]

In [37]:
df_fltr["weekday"]=df_fltr["ts"].dt.weekday
#df_fltr["weekday"]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
time_data=df_fltr.loc[:,['ts','hour','day','weekofyear','month','year','weekday']]
#time_data

In [39]:
df_fltr.loc[:]['ts'].dt.weekday

2     3
4     3
5     3
6     3
7     3
8     3
9     3
10    3
12    3
13    3
14    3
Name: ts, dtype: int64

In [40]:
column_labels=['timestamp','hour','day','weekofyear','month','year','weekday']

In [41]:
time_data.columns=column_labels
#time_data

In [42]:
#time_data = ()
#column_labels = ()

In [43]:
time_df = time_data
time_df.head()

,timestamp,hour,day,weekofyear,month,year,weekday
2,1970-01-01 00:25:41.106106796,0,1,1,1,1970,3
4,1970-01-01 00:25:41.106352796,0,1,1,1,1970,3
5,1970-01-01 00:25:41.106496796,0,1,1,1,1970,3
6,1970-01-01 00:25:41.106673796,0,1,1,1,1970,3
7,1970-01-01 00:25:41.107053796,0,1,1,1,1970,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [44]:
time_table_insert

'INSERT INTO time(start_time,hour,day,week,month,year,weekday)values(%s,%s,%s,%s,%s,%s,%s);\n'

In [45]:
time_df.dtypes

timestamp     datetime64[ns]
hour                   int64
day                    int64
weekofyear             int64
month                  int64
year                   int64
weekday                int64
dtype: object

In [46]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [47]:
user_df = df_log.loc[:,['user_id','first_name','last_name','gender','level']]

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [48]:
user_table_insert


'INSERT INTO users(user_id,first_name,last_name,gender,level)values(%s,%s,%s,%s,%s);\n'

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [50]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [51]:
df_log.columns


Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId', 'ts_val'],
      dtype='object')

In [52]:
songplay_data=df_fltr.loc[:,['ts','userId', 'level', 'song', 'artist','sessionId','location','userAgent']]
#songplay_data

In [53]:
#song_select = ("""SELECT songs.song_id,artists.artist_id FROM songs inner join artists on songs.artist_id=artists.artist_id where songs.title ='%s' and artists.name='%s' and songs.duration='%s';""")

In [54]:
song_select

'SELECT songs.song_id,artists.artist_id FROM songs inner join artists on songs.artist_id=artists.artist_id where songs.title =%s and artists.name=%s and songs.duration=%s;'

In [55]:
for index, row in df_log.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    
    # insert songplay record
    songplay_data=(str(row.ts_val),str(row.userId), str(row.level),str(songid), str(artistid), str(row.sessionId),str(row.location),str(row.userAgent))
    print(songplay_data)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

('1970-01-01 00:25:41.105830796', '39', 'free', 'None', 'None', '38', 'San Francisco-Oakland-Hayward, CA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('1970-01-01 00:25:41.106106796', '8', 'free', 'None', 'None', '139', 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"')
('1970-01-01 00:25:41.106106796', '8', 'free', 'None', 'None', '139', 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"')
('1970-01-01 00:25:41.106132796', '8', 'free', 'None', 'None', '139', 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"')
('1970-01-01 00:25:41.106352796', '8', 'free', 'None', 'None', '139', 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Win

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [56]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.